In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import regex as re
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', 500)

### **1. Inladen data**

In [ ]:
df = pd.read_csv('/mnt/data/jmaathuis/no_shows/no_show_all_apps_run2.csv', sep=';', parse_dates=['INVOERDAT', 'STARTDATEPLAN'], 
                dtype={'PATIENTNR': 'int64', 'MERGED': 'int64', 'GESLACHT': str, 'POSTOCDE': str, 'WOONPLAATS': str, #'LEEFTIJD': 'int64', 
                        'STARTTIMEPLAN': str, 'AANKOMST': str, 'AGENDA': str, 'SUBAGENDA': str, 'SPECCODE': str, 'TARAFD': str,
                        'LOCATIONID': str, 'DESCRIPTION': str, 'IsVoldaan': str, 'AfspraakstatusKey': 'int64', 'CONSTYPE': str, 'CODE': str})
   
#0 for show, 1 for no-show
df['no_show'] = df['AfspraakstatusKey'].isin([6, 8]).astype(int)

In [ ]:
df['IsVoldaan'].value_counts(dropna=False)

In [ ]:
df['AfspraakstatusKey'].value_counts()

De dataset kan in twee datasets worden gesplitst:  
1. de train test dataset
2. de historie dataset

De train test dataset bestaat uit data van 2020 tot en met 2023.  
De historie dataset bestaat uit de data die nodig is om bepaalde features te maken in de train test dataset. Denk hierbij bijvoorbeeld aan aantal eerdere no-shows.

In [ ]:
df_tt = df[df['STARTDATEPLAN'] >= '2020-01-01']
df_historie = df.loc[~df.index.isin(df_tt.index)]

# check:
print(len(df) == (len(df_tt) + len(df_historie)))

# old df out of memory
del df

In [ ]:
# calcualte the start_date of each historie (-5 years)
df_tt.loc[:,'START_HISTORIE'] = (df_tt['STARTDATEPLAN'] - pd.DateOffset(years=5))

### **2. Data exploratie**

Twee type features:
1. normale features die we rechtstreeks kunnen gebruiken
2. engineerd features (features die nog gemaakt moeten worden)

In [ ]:
class DataInspector():

    def __init__(self, data: pd.Series, column: str):
        self.data = data.copy()
        self.column = column
    
    def count_values(self):
        return self.data[self.column].value_counts(dropna=False).sort_index()


class NumericalDataInspector(DataInspector):

    def show_distributions(self):
        numerical_data = self.data[self.column]

        print(f'Mean:   {numerical_data.mean():.2f}')
        print(f'Median: {numerical_data.median():.2f}')
        print(f'Min:    {numerical_data.min():.0f}')
        print(f'Max:    {numerical_data.max():.0f}')
    
    def plot_distributions(self, x_label, bins):
        # - data wrangling - #
        df = self.data

        values_shows = df.loc[df['no_show'] == 0, self.column]
        values_no_shows = df.loc[df['no_show'] == 1, self.column]

        # calculate the percentages of the groups per bin 
        df['bins'] = pd.cut(df[self.column], bins, include_lowest=True)
        percs = df.groupby('bins')['no_show'].value_counts(normalize=True).unstack() * 100

        # - plotting - #
        fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(self.column + '\n\n')

        # -- histogram -- #
        ax1.hist(values_shows, label='Shows', bins=bins, alpha=0.4)
        ax1.hist(values_no_shows, label='No-Shows', bins=bins, alpha=0.4)
        ax1.set_xlabel(x_label)
        ax1.grid(alpha=0.2)
        ax1.set_ylabel('Frequency')
        ax1.set_title('Histogram')

        # -- normalized histogram -- #
        ax2.hist(values_shows, density=True, bins=bins, alpha=0.4)
        ax2.hist(values_no_shows, density=True, bins=bins, alpha=0.4)
        ax2.set_xlabel(x_label)
        ax2.grid(alpha=0.2)
        ax2.set_ylabel('Density')
        ax2.set_title('Normalized histogram')

        # -- boxplot -- #
        bp = ax3.boxplot([values_shows.dropna().tolist(), values_no_shows.dropna().tolist()], vert=False, patch_artist=True)

        boxplot_colors = ['C0', 'C1']
        for patch, color in zip(bp['boxes'], boxplot_colors):
            patch.set_facecolor(color)
        for box in bp['boxes']:
            box.set_alpha(0.5)
        for median in bp['medians']:
            median.set(color ='black')
        for flier in bp['fliers']:
            flier.set(marker ='D', markeredgecolor ='black', alpha=0.1)

        ax3.set_yticklabels(['Shows', 'No-Shows'])
        ax3.set_xlabel(x_label)  
        ax3.grid(alpha=0.2)
        ax3.set_title('Boxplot')
        
        # -- percentage chart -- #
        ax4.bar(percs.index.astype(str), percs[0], alpha=0.4)
        ax4.bar(percs.index.astype(str), percs[1], bottom=percs[0], alpha=0.4)
        ax4.tick_params(axis='x', labelrotation=90)
        ax4.set_ylabel('Percentage')
        ax4.set_xlabel('Bin')
        ax4.set_title('Percentage chart')
        ax4.set_ylim(0, 110)
        
        # plot
        fig.legend(bbox_to_anchor=(0.55, 0.95))
        plt.tight_layout()
        plt.show()


class CategoricalDataInspector(DataInspector):

    def plot_distributions(self, x_label, labels=None):

        # - data wrangling - #
        values = self.count_values()
        df = self.data
        
        values_per_label = df.groupby([self.column, 'no_show'], dropna=False)[self.column].value_counts(dropna=False)
        values_per_label = values_per_label.unstack().T.fillna(0)
        values_shows = values_per_label.loc[0].astype(int)
        values_no_shows = values_per_label.loc[1].astype(int)

        perc_shows = values_shows/(values_shows + values_no_shows) * 100
        perc_no_shows = values_no_shows/(values_shows + values_no_shows) * 100

        # - plotting - #
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        plt.suptitle(self.column + '\n\n')
        if labels == None:
            labels = values.keys().astype(str)

        # -- histogram -- #
        ax1.bar(labels, values_shows, alpha=0.4, label='Shows')
        ax1.bar(labels, values_no_shows, alpha=0.4, label='No-Shows')

        ax1.set_xlabel(x_label)
        ax1.grid(alpha=0.2)
        ax1.tick_params(axis='x', labelrotation=90)
        ax1.set_ylabel('Frequency')
        ax1.set_title('Histogram')

        # -- percentage barchart -- #
        ax2.bar(labels, perc_shows, alpha=0.4)
        bars = ax2.bar(labels, perc_no_shows, bottom=perc_shows, alpha=0.4)
        ax2.bar_label(bars, round(perc_no_shows), fontsize=9, rotation=90)

        ax2.set_xlabel(x_label)
        ax2.grid(alpha=0.2)
        ax2.tick_params(axis='x', labelrotation=90)
        ax2.set_ylabel('Density')
        ax2.set_ylim(0, 115)
        ax2.set_title('Percentage chart')

        fig.legend(bbox_to_anchor=(0.55, 0.93))
        plt.tight_layout()
        plt.show()


class DateDataInspector(DataInspector):
    
    def plot_distribution(self, x_label, bins):

        # - data wrangling - #
        df = self.data
        df_dates = df[self.column]

        date_min = df_dates.min()
        date_max = df_dates.max()

        values_shows = df.loc[df['no_show'] == 0, self.column]
        values_no_shows = df.loc[df['no_show'] == 1, self.column]

        # - plotting - #
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        fig.suptitle(self.column + '\n\n')

        # -- histogram -- #
        ax1.hist(values_shows, label='Shows', density=False, alpha=0.4, bins=bins, range=(date_min, date_max))
        ax1.hist(values_no_shows, label='No-Shows', density=False, alpha=0.4, bins=bins, range=(date_min, date_max))
        ax1.set_title(f'Histogram')
        ax1.set_xlabel(x_label)
        ax1.grid(alpha=0.2)
        ax1.set_ylabel('Frequency')

        # -- normalized histogram -- #
        ax2.hist(values_shows, density=True, alpha=0.4, bins=bins, range=(date_min, date_max))
        ax2.hist(values_no_shows, density=True, alpha=0.4, bins=bins, range=(date_min, date_max))
        ax2.set_xlabel(x_label)
        ax2.set_title('Normalized histogram')
        ax2.grid(alpha=0.2)
        ax2.set_ylabel('Density')

        fig.legend(bbox_to_anchor=(0.55, 0.93))
        plt.tight_layout()
        plt.show()

class TimeDataInspector(NumericalDataInspector):

    def __init__(self, data, column):
        super().__init__(data, column)
        self.data[self.column] = self.cat_time_to_hours(data[column])

    def cat_time_to_hours(self, time_data):
        time_data = pd.to_datetime(time_data,format= '%H:%M').dt.hour
        return(time_data)

class DistanceCalculator:

    def __init__(self, zip_code: str):
        ''' Zip code in the following format: "AAAA12" or "AAAA 12" '''        
        self.gelocator = Nominatim(user_agent="DistanceCalculator")
        self.start_coords = self.get_coordinates(zip_code)
    
    def evaluate_zip_code(self, zip_code):
        zip_code = zip_code.upper()
        zip_code = ''.join(zip_code.split(' '))

        if not re.match('^\d{4}[A-Z]{2}$', zip_code):
            raise ValueError('Zip code does not have the right format')
        
        return zip_code

    
    def get_coordinates(self, zip_code):
        zip_code = self.evaluate_zip_code(zip_code)
        location = self.gelocator.geocode(zip_code)

        return(location.longitude, location.latitude)
    
    def get_distance(self, zip_code):
        end_coords = self.get_coordinates(zip_code)
        distance = geodesic(self.start_coords, end_coords).kilometers

        return distance
    


#### **2.1 Normale features**

##### **2.1.1 `Patient nummer`**

In [ ]:
df_tt.PATIENTNR.nunique() / len(df_tt) * 100

1342398 verschillende patieten waarvan 16% uniek is.   
Veel dubbeling dus.

##### **2.1.2 `Merged`**

In [ ]:
df_tt.MERGED

In [ ]:
mergedDataInspector = CategoricalDataInspector(df_tt, 'MERGED')
mergedDataInspector.count_values()

In [ ]:
mergedDataInspector.plot_distributions('Merged', labels=['No', 'Yes'])

There does not seem te be a difference in patients that are merged based on the target variable

##### **2.1.3 `Geslacht`**

In [ ]:
df_tt['GESLACHT']

In [ ]:
df_tt['GESLACHT'].value_counts()

In [ ]:
df_tt[df_tt['GESLACHT'] == 'O']

Deze patienten heb ik opgezocht in de database en zijn test patienten, dus ze kunnnen uit de dataset.

In [ ]:
df_tt = df_tt[df_tt['GESLACHT'] != 'O']

In [ ]:
geslachtDataInspector = CategoricalDataInspector(df_tt, 'GESLACHT')
geslachtDataInspector.count_values()

In [ ]:
geslachtDataInspector.plot_distributions('Geslacht')

##### **2.1.4 `Leeftijd`**


In [ ]:
df_tt.LEEFTIJD

In [ ]:
leeftijdDataInspector = NumericalDataInspector(df_tt, 'LEEFTIJD')

In [ ]:
leeftijdDataInspector.count_values()

In [ ]:
leeftijdDataInspector.show_distributions()

In [ ]:
leeftijdDataInspector.plot_distributions('Leeftijd', bins=np.arange(0,126, 5))

##### **2.1.5 `Datum inplannen afspraak`**


In [ ]:
df_tt.INVOERDAT

In [ ]:
invoerdatDataInspector = DateDataInspector(df_tt, 'INVOERDAT')
invoerdatDataInspector.plot_distribution('Datum inplannen afspraak', 40)

##### **2.1.6 `Datum afspraak`**


In [ ]:
invoerdatDataInspector = DateDataInspector(df_tt, 'STARTDATEPLAN')
invoerdatDataInspector.plot_distribution('Datum inplannen afspraak', 30)
# df_tt

##### **2.1.7 `Tijd afspraak`**

In [ ]:
df_tt['STARTTIMEPLAN']

In [ ]:
startTimeDataInspector = TimeDataInspector(df_tt, 'STARTTIMEPLAN')
startTimeDataInspector.count_values()

In [ ]:
startTimeDataInspector.plot_distributions(x_label='Tijd afspraak', bins=np.arange(0, 25,1))

Dal tijdens lunch tijd 

##### **2.1.8 `Aankomst tijd`**

In [ ]:
df_tt['AANKOMST']

In [ ]:
aankomstTimeDataInspector = TimeDataInspector(df_tt, 'AANKOMST')
aankomstTimeDataInspector.count_values()

Heel veel NaN's waarschijnijk doordat de aankomst tijd niet wordt ingevuld als de patient een no-show is

In [ ]:
aankomstTimeDataInspector.plot_distributions('Aankomst tijd', bins=np.arange(0, 25, 1))

Bijna geen data van no-show patienten, omdat daarvan geen aankomst tijd is ingevuld

##### **2.1.9 `Agenda afspraak`**

In [ ]:
df_tt['AGENDA']

In [ ]:
agendaDataInspector = CategoricalDataInspector(df_tt, 'AGENDA')
agendaDataInspector.count_values()

In [ ]:
agendaDataInspector.plot_distributions('Agenda afpsraak')

##### **2.1.10 `Specialisme`**

In [ ]:
specialismDataInspector = CategoricalDataInspector(df_tt, 'SPECCODE')
specialismDataInspector.count_values()

In [ ]:
specialismDataInspector.plot_distributions(x_label='Specialisme')

##### **2.1.11 `Consult type`**

In [ ]:
df_tt['CONSTYPE']

In [ ]:
consultDataInspector = CategoricalDataInspector(df_tt, 'CONSTYPE')
consultDataInspector.count_values()

In [ ]:
consultDataInspector.plot_distributions('Consult type')

##### **2.1.12 `Code`**

In [ ]:
df_tt['CODE']

In [ ]:
codeDataInspector = CategoricalDataInspector(df_tt, 'CODE')
codeDataInspector.count_values()

heel veel verschillende cataegorien. Misschien groeperen, of niet gebruiken i.v.m. ruis. 

#### **2.2 Engineerd features**

In [ ]:
# feature engineering is done using a seperate script, so reload data
df_tt = pd.read_csv('/mnt/data/jmaathuis/no_shows/no_shows_pp_v5.csv')

##### **2.2.1 `Locatie`**

In [ ]:
loc_keep = ['ZGT locatie Almelo', 'ZGT locatie Hengelo', 'Polikliniek Verloskunde Almelo', 'Obesitas centrum Hengelo ZGT', 'Oncologisch centrum Hengelo', 'Behandelcentrum Almelo', 'Slaapcentrum Hengelo', 'Behandelcentrum Hengelo', np.nan]
df_tt = df_tt[df_tt['DESCRIPTION'].isin(loc_keep)]
df_tt['DESCRIPTION'].value_counts(dropna=False).sort_values()

In [ ]:
locationDataInspector = CategoricalDataInspector(df_tt, 'LOCATIE')
locationDataInspector.count_values()

In [ ]:
locationDataInspector.plot_distributions('Locatie')

#### **2.2.2 `Afstand`**

In [ ]:
df_tt['AFSTAND']

In [ ]:
afstandDataInspector = NumericalDataInspector(df_tt, 'AFSTAND')
afstandDataInspector.show_distributions()

In [ ]:
afstandDataInspector.plot_distributions('Afstand (km)', bins=range(0, 366, 10))

#### **2.2.3 `Verschil inplannen en afspraak`** 

In [ ]:
dayDeltaDataInspector = NumericalDataInspector(df_tt, 'VerschilInplannenEnAfspraak')
dayDeltaDataInspector.show_distributions()

In [ ]:
dayDeltaDataInspector.plot_distributions('Verschil inplannen en afspraak (dagen)', bins=range(0, 820, 20))

Er zijn dus wat afspraken die later gepland zijn dan dat ze hebben plaats gevonden (veel SEH afspraken).   
Aangezien ik geen no-show kan voorspellen van afspraken in het verleden verwijder ik deze afspraken:   

** Deze zijn bij de re-run van dit notebook er al uitgehaald dus niet meer zichtbaar **

#### **2.2.3 `Stiptheid`**

Stiptheid is alsvolgt gedefinieerd: Gemiddelde verschil in tijd tussen aankomst en tijd afspraak

In [ ]:
df_tt['stiptheid']

In [ ]:
stiptheidDataInspector = NumericalDataInspector(df_tt, 'stiptheid')
stiptheidDataInspector.show_distributions()

In [ ]:
stiptheidDataInspector.plot_distributions('Stiptheid (min)', bins=range(-180, 180, 5))

mooie normaal verdeling.   
Zoals je kan zien zijn er meer no shows als de patient minder stipt is.  
Interessant, ben benieuwd wat de feature importance hiervan is.

#### **2.2.4 `Eerdere no shows`**

In [ ]:
df_tt['num_no_shows']

In [ ]:
numNoShowsDataInspector = NumericalDataInspector(df_tt, 'num_no_shows')
numNoShowsDataInspector.show_distributions() 

In [ ]:
numNoShowsDataInspector.plot_distributions(x_label='Aantal eerdere no-shows', bins=range(0,44, 1))

#### **2.2.5 `Percentage eerdere no shows`**

In [ ]:
df_tt['perc_no_shows']

In [ ]:
percNoShowsDataInspector = NumericalDataInspector(df_tt, 'perc_no_shows')
percNoShowsDataInspector.show_distributions()

In [ ]:
percNoShowsDataInspector.plot_distributions(x_label='Eerdere no shows (%)', bins=np.arange(0, 1.01, 0.025))